In [1]:
import sqlite3

# Create the database
conn = sqlite3.connect('balliq.db')
c = conn.cursor()

# Create the Users table
c.execute('''CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    password TEXT NOT NULL,
    team_name TEXT NOT NULL,
    budget REAL NOT NULL CHECK(budget >= 0 AND budget <= 100),
    points INTEGER DEFAULT 0 CHECK(points >= 0)
)''')

# Create the Players Fantasy table
c.execute('''CREATE TABLE IF NOT EXISTS players_fantasy (
    player_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    team TEXT NOT NULL,
    position TEXT NOT NULL CHECK(position IN ('Goalkeeper', 'Defender', 'Midfielder', 'Forward')),
    price REAL NOT NULL CHECK(price >= 0),
    price_evolution REAL DEFAULT 0,
    points_per_game REAL DEFAULT 0,
    form_rank INTEGER CHECK(form_rank >= 0),
    total_fantasy_points INTEGER DEFAULT 0,
    next_week INTEGER DEFAULT 0 CHECK(next_week >= 0),
    expected_points_next_game REAL DEFAULT 0
)''')

# Create the Player Stats table
c.execute('''CREATE TABLE IF NOT EXISTS player_stats (
    player_id INTEGER PRIMARY KEY,
    goals INTEGER DEFAULT 0 CHECK(goals >= 0),
    assists INTEGER DEFAULT 0 CHECK(assists >= 0),
    yellow_cards INTEGER DEFAULT 0 CHECK(yellow_cards >= 0),
    red_cards INTEGER DEFAULT 0 CHECK(red_cards >= 0),
    penalties_defended INTEGER DEFAULT 0 CHECK(penalties_defended >= 0),
    own_goals INTEGER DEFAULT 0 CHECK(own_goals >= 0),
    clean_sheets INTEGER DEFAULT 0 CHECK(clean_sheets >= 0),
    saves INTEGER DEFAULT 0 CHECK(saves >= 0),
    starts INTEGER DEFAULT 0 CHECK(starts >= 0),
    penalties_missed INTEGER DEFAULT 0 CHECK(penalties_missed >= 0),
    FOREIGN KEY (player_id) REFERENCES players_fantasy(player_id) ON DELETE CASCADE
)''')

# Create the User Team table
c.execute('''CREATE TABLE IF NOT EXISTS user_team (
    user_id INTEGER NOT NULL,
    player_id INTEGER NOT NULL,
    starting_eleven INTEGER DEFAULT 0 CHECK(starting_eleven IN (0, 1)),
    on_team INTEGER DEFAULT 0 CHECK(on_team IN (0, 1)),
    FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE,
    FOREIGN KEY (player_id) REFERENCES players_fantasy(player_id) ON DELETE CASCADE,
    PRIMARY KEY (user_id, player_id)
)''')

# Create the Fixtures table
c.execute('''CREATE TABLE IF NOT EXISTS fixtures (
    week INTEGER NOT NULL,
    date TEXT NOT NULL,
    home_team TEXT NOT NULL,
    away_team TEXT NOT NULL,
    PRIMARY KEY (week, home_team, away_team)
)''')

# Adding indexes for better performance
c.execute('''CREATE INDEX IF NOT EXISTS idx_user_team_user_id ON user_team(user_id)''')
c.execute('''CREATE INDEX IF NOT EXISTS idx_user_team_player_id ON user_team(player_id)''')
c.execute('''CREATE INDEX IF NOT EXISTS idx_player_stats_player_id ON player_stats(player_id)''')
c.execute('''CREATE INDEX IF NOT EXISTS idx_players_fantasy_next_week ON players_fantasy(next_week)''')
c.execute('''CREATE INDEX IF NOT EXISTS idx_fixtures_week ON fixtures(week)''')

# Commit changes and close the connection
conn.commit()
conn.close()


In [2]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import ChatOpenAI
import sqlite3
import pandas as pd
from pydantic import BaseModel, Field, model_validator, ValidationError
from typing import Optional
from datetime import date
import numpy as np


In [101]:
from dotenv import load_dotenv
import os

# Load the OpenAI API key from the .env file
load_dotenv()
api_key = os.getenv("api_key")
if api_key is None:
    raise ValueError("The api_key environment variable is not set.")

# Generate synthetic data for User

In [3]:
# Users Table
class User(BaseModel):
    username: str
    email: str
    password: str
    team_name: str
    budget: float = Field(..., ge=0, le=100.0)
    points: int = Field(0, ge=0) 

In [103]:
examples = [
    {
        "example": """username: rmaia44, email: rmaia2003@gmail.com,
        password: maiar04, team_name: SadioMane, budget: 100.0, points: 0"""
    },
    {
        "example": """username: mmendes04, email: miguelmendes04@gmail.com,
        password: mmendes-04, team_name: Hater do Amorim, budget: 100.0, points: 0"""
    },
    {
        "example": """username: jocajoca77, email: jferreira@gmail.com,
        password: joaof005, team_name: Joca, budget: 100.0, points: 0"""
    }
]

In [6]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

In [7]:
llm = ChatOpenAI(api_key=api_key, model="gpt-3.5-turbo", temperature=1)

In [8]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=User,
    llm=llm,
    prompt=prompt_template,
)

In [9]:
synthetic_results = synthetic_data_generator.generate(
    subject="users",
    extra = "Don't repeat anything. Try to be as creative as possible",
    runs=10,
)

In [7]:
synthetic_results

[User(username='paulinha92', email='paulinhacosta92@yahoo.com', password='costap21', team_name='RonaldoForever', budget=100.0, points=0),
 User(username='lucarvalho01', email='lucascarvalho01@hotmail.com', password='carvalho123', team_name='TeamCarvalho', budget=95.0, points=10),
 User(username='sydneyroyals', email='sydneyroyals22@gmail.com', password='royal2022', team_name='Sydney Royals', budget=98.5, points=5),
 User(username='marinaTheExplorer', email='marinaexplorer85@gmail.com', password='oceans1234', team_name='OceanAdventures', budget=80.0, points=15),
 User(username='adventureLover77', email='adventure77lover@yahoo.com', password='explore2022', team_name='Adventure Seekers', budget=82.0, points=8),
 User(username='wildernessWanderer', email='wildwanderer@gmail.com', password='wild2022', team_name='Wilderness Wanderers', budget=90.5, points=10),
 User(username='explorerExtraordinaire', email='extraordinaireexplorer@gmail.com', password='amazingexplore123', team_name='Fantastic

In [4]:
# Just copy and pasted the print of the previous cell to not have to generate new data again
synthetic_results = [User(username='paulinha92', email='paulinhacosta92@yahoo.com', password='costap21', team_name='RonaldoForever', budget=100.0, points=0),
 User(username='lucarvalho01', email='lucascarvalho01@hotmail.com', password='carvalho123', team_name='TeamCarvalho', budget=95.0, points=10),
 User(username='sydneyroyals', email='sydneyroyals22@gmail.com', password='royal2022', team_name='Sydney Royals', budget=98.5, points=5),
 User(username='marinaTheExplorer', email='marinaexplorer85@gmail.com', password='oceans1234', team_name='OceanAdventures', budget=80.0, points=15),
 User(username='adventureLover77', email='adventure77lover@yahoo.com', password='explore2022', team_name='Adventure Seekers', budget=82.0, points=8),
 User(username='wildernessWanderer', email='wildwanderer@gmail.com', password='wild2022', team_name='Wilderness Wanderers', budget=90.5, points=10),
 User(username='explorerExtraordinaire', email='extraordinaireexplorer@gmail.com', password='amazingexplore123', team_name='Fantastic Explorers', budget=85.0, points=12),
 User(username='oceanAdventurer23', email='adventureocean23@gmail.com', password='deepsea2023', team_name='Ocean Voyagers', budget=78.3, points=9),
 User(username='mountainTrailBlazer', email='mountainblazer22@gmail.com', password='hikingtrails2022', team_name='Summit Seekers', budget=88.7, points=15),
 User(username='techSavvyGuru', email='gurutechmaster@gmail.com', password='codeNinja2023', team_name='Digital Innovators', budget=92.5, points=18)]

### Upload this new data to the database

In [5]:
conn = sqlite3.connect('balliq.db', check_same_thread=False)
c = conn.cursor()

for i, result in enumerate(synthetic_results):
    c.execute("INSERT INTO users (user_id, username, email, password, team_name, budget, points) VALUES (?, ?, ?, ?, ?, ?, ?)",
            (i, result.username, result.email, result.password, result.team_name, result.budget, result.points))

conn.commit()
conn.close()

In [6]:
conn = sqlite3.connect('balliq.db', check_same_thread=False)
c = conn.cursor()


c.execute("INSERT INTO users (user_id, username, email, password, team_name, budget, points) VALUES (?, ?, ?, ?, ?, ?, ?)",
         (11, "Goat", "goat123@gmail.com", "goatedcr7", "Ronaldo da Fantasy", 50, 100))

c.execute("INSERT INTO users (user_id, username, email, password, team_name, budget, points) VALUES (?, ?, ?, ?, ?, ?, ?)",
         (10000, "Test", "test@gmail.com", "test", "Test", 100, 0))

conn.commit()
conn.close()

# Upload data to the database from a csv

In [7]:
teams = {
    "Arsenal": "Arsenal",
    "Aston Villa": "Aston Villa",
    "Bournemouth": "Bournemouth",
    "Brentford": "Brentford",
    "Brighton": "Brighton",
    "Chelsea": "Chelsea",
    "Crystal Palace": "Crystal Palace",
    "Everton": "Everton",
    "Fulham": "Fulham",
    "Ipswich": "Ipswich Town",
    "Leicester": "Leicester City",
    "Liverpool": "Liverpool",
    "Man City": "Manchester City",
    "Man Utd": "Manchester United",
    "Newcastle": "Newcastle United",
    "Nott'm Forest": "Nottingham Forest",
    "Southampton": "Southampton",
    "Spurs": "Tottenham",
    "West Ham": "West Ham",
    "Wolves": "Wolves"
}


In [8]:
fantasy_data = pd.read_csv("players.csv")
fantasy_data["team"] = fantasy_data["team"].map(teams)
fantasy_data['now_cost'] = fantasy_data['now_cost']/10
fantasy_data['cost_change_start'] = fantasy_data['cost_change_start']/10
def map_position(position):
    mapping = {
        'MID': 'Midfielder',
        'DEF': 'Defender',
        'FWD': 'Forward',
        'GKP': 'Goalkeeper'
    }
    return mapping.get(position, position) 
fantasy_data["position"] = fantasy_data['position'].apply(lambda x: map_position(x))
print(fantasy_data["position"].unique())
# Connect to the database
conn = sqlite3.connect('balliq.db')
c = conn.cursor()



for index, row in fantasy_data.iterrows():
    c.execute("""
        INSERT INTO players_fantasy
        (player_id, name, team, position, price, price_evolution, points_per_game, form_rank, total_fantasy_points, next_week, expected_points_next_game)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
        (row["id"],
        row['name'],
        row['team'],
        row['position'],
        row['now_cost'],
        row['cost_change_start'],
        row["points_per_game"],
        row["form_rank_type"],
        row['total_points'],
        20,
        row['ep_next']
    ))
    
    c.execute("""
        INSERT INTO player_stats
        (player_id, goals, assists, yellow_cards, red_cards, penalties_defended, own_goals, clean_sheets, saves, starts, penalties_missed)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
        (row["id"],
        row['goals_scored'],
        row['assists'],
        row['yellow_cards'],
        row['red_cards'],
        row["penalties_saved"],
        row["own_goals"],
        row['clean_sheets'],
        row['saves'],
        row['starts'],
        row['penalties_missed']))

conn.commit()
conn.close()

['Midfielder' 'Forward' 'Defender' 'Goalkeeper']


# Generate Data for User Team table


In [9]:
class UserTeam(BaseModel):
    user_id: int = Field(..., ge= 0)
    player_id: int = Field(..., ge=1, le=678)
    starting_eleven: int = Field(..., ge=0, le=1)  

In [10]:
users_team_synthetic = []
for usern in range(1,11):
    available_options = [i for i in range(1,679)]
    for _ in range(5):
        player = np.random.choice(available_options)
        users_team_synthetic.append(UserTeam(user_id=usern, player_id=player, starting_eleven= np.random.randint(0,2)))
        available_options.remove(player)

In [11]:
len(users_team_synthetic)

50

### Add this new data to the database

In [12]:
conn = sqlite3.connect('balliq.db', check_same_thread=False)
c = conn.cursor()

for result in users_team_synthetic:
    c.execute("INSERT INTO user_team (user_id, player_id, starting_eleven, on_team) VALUES (?, ?, ?, ?)",
            (result.user_id, result.player_id, result.starting_eleven, 1))

conn.commit()
conn.close()

In [13]:
#Handmade user insert

user11 = [UserTeam(user_id=11, player_id=1, starting_eleven= 1),
          UserTeam(user_id=11, player_id= 2, starting_eleven= 0),
          UserTeam(user_id=11, player_id=3, starting_eleven= 1),
          UserTeam(user_id=11, player_id=4, starting_eleven= 0),
          UserTeam(user_id=11, player_id=5, starting_eleven= 1),
          UserTeam(user_id=11, player_id=6, starting_eleven= 1),
          UserTeam(user_id=11, player_id=7, starting_eleven= 0),
          UserTeam(user_id=11, player_id=8, starting_eleven= 1),
          UserTeam(user_id=11, player_id=9, starting_eleven= 1),
          UserTeam(user_id=11, player_id=10, starting_eleven= 0),
          UserTeam(user_id=11, player_id=11, starting_eleven= 1),
          UserTeam(user_id=11, player_id=12, starting_eleven= 0),
          UserTeam(user_id=11, player_id=13, starting_eleven= 1),
          UserTeam(user_id=11, player_id=14, starting_eleven= 0),
          UserTeam(user_id=11, player_id=15, starting_eleven= 0),
          UserTeam(user_id=11, player_id=16, starting_eleven= 1),
          UserTeam(user_id=11, player_id=17, starting_eleven= 0),
          UserTeam(user_id=11, player_id=18, starting_eleven= 1),
          UserTeam(user_id=11, player_id=19, starting_eleven= 1),
          UserTeam(user_id=11, player_id=20, starting_eleven= 0),
          UserTeam(user_id=11, player_id=21, starting_eleven= 0),
          UserTeam(user_id=11, player_id=22, starting_eleven= 0),
          UserTeam(user_id=11, player_id=23, starting_eleven= 0),
          UserTeam(user_id=11, player_id=24, starting_eleven= 0),
          UserTeam(user_id=11, player_id=25, starting_eleven= 0),
          UserTeam(user_id=11, player_id=26, starting_eleven= 0),
]

In [14]:
#Handmade user insert
conn = sqlite3.connect('balliq.db', check_same_thread=False)
c = conn.cursor()

for result in user11:
    c.execute("INSERT INTO user_team (user_id, player_id, starting_eleven, on_team) VALUES (?, ?, ?, ?)",
            (result.user_id, result.player_id, result.starting_eleven, 1))

conn.commit()
conn.close()

# Upload data to Fixtures from csv

In [15]:
fixtures_data = pd.read_csv("epl-fixtures-2025.csv")

conn = sqlite3.connect('balliq.db', check_same_thread=False)
c = conn.cursor()

for index, row in fixtures_data.iterrows():
        c.execute("""
                INSERT INTO fixtures
                (week, date, home_team, away_team)
                VALUES (?, ?, ?, ?)""",
                (row["week"],
                row['date'],
                row['home'],
                row['away'],
                ))

conn.commit()
conn.close()